# Countries Explorative Data Analysis (EDA)

In [3]:
import csv
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import requests
import seaborn as sns
import time

## Overview
The data contains information about 186 countries.
The data can help analyze the population, area, and different details about the countries

## Questions for Analysis
1. What is the population distribution?
2. What is the most and least populated country?
3. What is the most and least populated continent?
4. What is the country with the largest and smallest average area?
5. What is the continent with the largest and smallest average area?
6. What is the country with the most inequality of wealth?
7. What is the continent with the most/least equality of wealth?
8. What is the most spoken language in the world?
9. How many countries drive on the left side?
10. How many countries are still not independant?
11. What is the continent with the most countries?
12. What country got his independence last?

## About Dataset

The dataset contains information about all the 250 delegations in the world. The datasetcomes from two different APIs and websites
- REST Countries API
- Restful Countries API

### Columns description
- ```a```
- b
- c
- d
- e

## Data Collection

### Create CSV file from first API

In [9]:
# Handle ChunkedEncodingError by adding 'try except' and adding 1 second pause between each attempt
for attempt in range(5):
    country_list = []
    
    try:
        response = requests.get('https://restcountries.com/v3.1/all')
        
        if response.status_code != 200:
            print('Failed to find data')

        countriesJson = json.loads(response.content)

        # Collect relevant columns
        for item in countriesJson:
            keys = item.keys()
            country = {
                'name': item['name']['common'],
                'cca3': item['cca3'],
                'continent' : item['continents'][0],
                'area': item['area'],
                'car_side': item['car']['side'],
                'gini': list(item['gini'].values())[0] if 'gini' in keys else None,
                'languages': ', '.join(list(item['languages'].values())) if 'languages' in keys else None
            }
            country_list.append(country)

        header = country_list[0].keys()

        # Create/replace CSV file
        with open('Data/countries1.csv', 'w', newline='') as output_file:
            dict_writer = csv.DictWriter(output_file, header)
            dict_writer.writeheader()
            dict_writer.writerows(country_list)
            output_file.close()
            print('CSV file', output_file.name, 'created')
    
        break
    except requests.exceptions.ChunkedEncodingError:
        time.sleep(1)
else:
    print('Failed to fetch data from REST countries API')

CSV file Data/countries1.csv created


### Create CSV file from second API

In [11]:
# Request data
api_key = '1787|qM666AtNiFBX3PjiuCjngNuBXh9VyRdggx94r1Hr'
url = 'https://restfulcountries.com/api/v1/countries'
headers = {'Accept': 'application/json', 'Authorization': 'Bearer' + ' ' + api_key}
                                
try:
    country_list2 = []
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print('Failed to find data')
    
    countries2Json = json.loads(response.content)['data']
    
    # Collect relevant columns
    for item in countries2Json:
        country = {
            'name': item['name'],
            'iso3': item['iso3'],
            'capital': item['capital'],
            'currency': item['currency'],
            'population': item['population']
        }
        country_list2.append(country)
    
    header = country_list2[0].keys()
    
    # Create/replace CSV file
    with open('Data/countries2.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, header)
        dict_writer.writeheader()
        dict_writer.writerows(country_list2)
        output_file.close()
        print('CSV file', output_file.name, 'created')
        
except:
    print('Failed to fetch data from Restful Countries API')

CSV file Data/countries2.csv created


### Create dataframe from 2 CSV files

In [13]:
# Get number of rows/columns of each dataframe
countries1 = pd.read_csv('Data/countries1.csv')
print('Countries 1 shape:', countries1.shape)

countries2 = pd.read_csv('Data/countries2.csv')
print('Countries 2 shape:', countries2.shape)

Countries 1 shape: (250, 7)
Countries 2 shape: (186, 5)


In [14]:
# Merge dataframes
countries = countries1.merge(right=countries2, how='inner', left_on='cca3', right_on='iso3')
print('Merged dataframe shape:', countries.shape)

Merged dataframe shape: (186, 12)


## Data Exploration

In [16]:
countries.head()

,name_x,cca3,continent,area,car_side,gini,languages,name_y,iso3,capital,currency,population
0,Grenada,GRD,North America,344.0,left,NaN,English,Grenada,GRD,St. George's,XCD,"112,742"
1,Switzerland,CHE,Europe,41284.0,right,33.1,"French, Swiss German, Italian, Romansh",Switzerland,CHE,Berne,CHF,"8,681,478"
2,Sierra Leone,SLE,Africa,71740.0,right,35.7,English,Sierra Leone,SLE,Freetown,SLL,"8,047,131"
3,Hungary,HUN,Europe,93028.0,right,29.6,Hungarian,Hungary,HUN,Budapest,HUF,"9,650,147"
4,Taiwan,TWN,Asia,36193.0,right,NaN,Chinese,Taiwan,TWN,Taipei,TWD,"23,834,918"


In [17]:
countries.tail()

,name_x,cca3,continent,area,car_side,gini,languages,name_y,iso3,capital,currency,population
181,Republic of the Congo,COG,Africa,342000.0,right,48.9,"French, Kikongo, Lingala",Congo,COG,Brazzaville,XAF,"5,576,861"
182,Belgium,BEL,Europe,30528.0,right,27.2,"German, French, Dutch",Belgium,BEL,Brussels,EUR,"11,610,784"
183,Israel,ISR,Asia,20770.0,right,39.0,"Arabic, Hebrew",Israel,ISR,Jerusalem,ILS,"8,713,402"
184,New Zealand,NZL,Oceania,270467.0,left,NaN,"English, Māori, New Zealand Sign Language",New Zealand,NZL,Wellington,NZD,"4,838,834"
185,Nicaragua,NIC,North America,130373.0,right,46.2,Spanish,Nicaragua,NIC,Managua,NIO,"6,658,149"


In [18]:
countries.sample(5)

,name_x,cca3,continent,area,car_side,gini,languages,name_y,iso3,capital,currency,population
154,Malta,MLT,Europe,316.0,left,28.7,"English, Maltese",Malta,MLT,Valletta,EUR,"442,037"
88,Marshall Islands,MHL,Oceania,181.0,right,NaN,"English, Marshallese",Marshall Islands,MHL,Majuro,USD,"59,359"
58,India,IND,Asia,3287590.0,left,35.7,"English, Hindi, Tamil",India,IND,New Delhi,INR,"1,385,753,930"
80,Saudi Arabia,SAU,Asia,2149690.0,right,NaN,Arabic,Saudi Arabia,SAU,Riyadh,SAR,"35,046,623"
44,Algeria,DZA,Africa,2381741.0,right,27.6,Arabic,Algeria,DZA,Algiers,DZD,"44,190,030"


In [19]:
countries.dtypes

name_x         object
cca3           object
continent      object
area          float64
car_side       object
gini          float64
languages      object
name_y         object
iso3           object
capital        object
currency       object
population     object
dtype: object

In [20]:
countries.describe()

,area,gini
count,1.860000e+02,159.000000
mean,7.033734e+05,38.397484
std,1.945260e+06,7.945741
min,2.020000e+00,24.600000
25%,2.777100e+04,32.850000
50%,1.375450e+05,37.100000
75%,5.457632e+05,42.850000
max,1.709824e+07,63.000000


In [21]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name_x      186 non-null    object 
 1   cca3        186 non-null    object 
 2   continent   186 non-null    object 
 3   area        186 non-null    float64
 4   car_side    186 non-null    object 
 5   gini        159 non-null    float64
 6   languages   186 non-null    object 
 7   name_y      186 non-null    object 
 8   iso3        186 non-null    object 
 9   capital     186 non-null    object 
 10  currency    186 non-null    object 
 11  population  186 non-null    object 
dtypes: float64(2), object(10)
memory usage: 17.6+ KB


There are only missing values in the ```gini``` column. We will not remove the rows.

## Data Preparation

In [35]:
# Drop the name column of the second dataframe
countries = countries.drop('name_y', axis=1).rename(columns={'name_x': 'name'})
countries.head()

,name,cca3,continent,area,car_side,gini,languages,iso3,capital,currency,population
0,Grenada,GRD,North America,344.0,left,NaN,English,GRD,St. George's,XCD,"112,742"
1,Switzerland,CHE,Europe,41284.0,right,33.1,"French, Swiss German, Italian, Romansh",CHE,Berne,CHF,"8,681,478"
2,Sierra Leone,SLE,Africa,71740.0,right,35.7,English,SLE,Freetown,SLL,"8,047,131"
3,Hungary,HUN,Europe,93028.0,right,29.6,Hungarian,HUN,Budapest,HUF,"9,650,147"
4,Taiwan,TWN,Asia,36193.0,right,NaN,Chinese,TWN,Taipei,TWD,"23,834,918"


In [37]:
# Drop the code column of the second dataframe
countries = countries.drop('iso3', axis=1).rename(columns={'cca3': 'code'})
countries.head()

,name,code,continent,area,car_side,gini,languages,capital,currency,population
0,Grenada,GRD,North America,344.0,left,NaN,English,St. George's,XCD,"112,742"
1,Switzerland,CHE,Europe,41284.0,right,33.1,"French, Swiss German, Italian, Romansh",Berne,CHF,"8,681,478"
2,Sierra Leone,SLE,Africa,71740.0,right,35.7,English,Freetown,SLL,"8,047,131"
3,Hungary,HUN,Europe,93028.0,right,29.6,Hungarian,Budapest,HUF,"9,650,147"
4,Taiwan,TWN,Asia,36193.0,right,NaN,Chinese,Taipei,TWD,"23,834,918"


In [43]:
# Convert population column from string to float
countries['population'] = countries['population'].str.replace(',', '').astype('float')
countries.dtypes

name           object
code           object
continent      object
area          float64
car_side       object
gini          float64
languages      object
capital        object
currency       object
population    float64
dtype: object

In [47]:
# Sort countries in alphabetical order
countries = countries.sort_values(by='name')
countries.head(5)

,name,code,continent,area,car_side,gini,languages,capital,currency,population
31,Afghanistan,AFG,Asia,652230.0,right,NaN,"Dari, Pashto, Turkmen",Kabul,AFN,39306195.0
162,Albania,ALB,Europe,28748.0,right,33.2,Albanian,Tirana,lek,2876490.0
44,Algeria,DZA,Africa,2381741.0,right,27.6,Arabic,Algiers,DZD,44190030.0
13,Andorra,AND,Europe,468.0,right,NaN,Catalan,Andorra la Vella,EUR,77317.0
139,Angola,AGO,Africa,1246700.0,right,51.3,Portuguese,Luanda,AOA,33312843.0


In [49]:
countries.tail()

,name,code,continent,area,car_side,gini,languages,capital,currency,population
168,Venezuela,VEN,South America,916445.0,right,44.8,Spanish,Caracas,VEF,28402272.0
121,Vietnam,VNM,Asia,331212.0,right,35.7,Vietnamese,Hanoi,VND,97702766.0
21,Yemen,YEM,Asia,527968.0,right,36.7,Arabic,Sanaa,YER,30110883.0
33,Zambia,ZMB,Africa,752612.0,left,57.1,English,Lusaka,ZMK,18609335.0
178,Zimbabwe,ZWE,Africa,390757.0,left,50.3,"Chibarwe, English, Kalanga, Khoisan, Ndau, Nor...",Harare,ZWL,14955711.0


## Data Analysis